In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt

import evosax

from typing import NamedTuple
from functools import partial
import math


In [34]:
import json
import drawsvg

In [3]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.receptance as r
import xylo.sweep as xs

import xylo.loss.receptance as xlr

import xylo.tuning
import xylo.database as db

In [4]:
def sweep_blank(wood, bar):
    sections = xylo.cut.none(bar)
    # sections.plot(to_scale = True)
    swp = xs.sweep(wood, bar, sections, t.sweep_default)
    # print(wood)
    print(swp.harmonics)
    # print(swp.harmonics / swp.harmonics[0])
    print("")

In [20]:
def make_bar(l, d):
  return t.BarProps(width = 0.032, depth = d, length = l, elements = 1, min_depth = d)

In [30]:
def sweep_find_length(wood, f, d, freqd = 1, mind = 0.0001):
  l = 0.2
  ld = 0.1
  lf = None
  while ld > mind:
    bar = make_bar(l, d)
    swp = xs.sweep(wood, bar, xylo.cut.none(bar), t.sweep_default)
    lf = swp.harmonics[0]
    if lf < f - freqd:
      l -= ld
    elif lf > f + freqd:
      l += ld
    else:
      return l, lf
    ld /= 2
  return l, lf

In [25]:
def convert_length1(n, coeffs = [0.48, 0.5]):
  orig_l = xylo.tuning.yamaha_YX500R.note_to_length(n)
  return orig_l * coeffs[0] + (orig_l ** 2) * coeffs[1]

def convert_length2(n, coeffs = [0.40, -3.8, -5]):
  orig_l = n / 1000
  return coeffs[0] + orig_l * coeffs[1] + (orig_l ** 2) * coeffs[2]

def convert_length3(n):
  return xylo.tuning.glock_F5_200.note_to_length(n)

def convert_length4(n):
  nn = (n - 59)
  return 0.196 - nn * 0.004 # - (nn * nn) * 0.0001
  # return xylo.tuning.glock_G5_200.note_to_length(n)

sum_lengths = 0
count = 0

# for d in [0.002, 0.006, 0.010]:
for d in [0.006]:
  print(f"*** depth {d} ***")
  # f2-c5, 2 1/2 octaves
  # or F5-C8, 
  # or F5-F7
  # for n in range(57, 82):
  # for n in range(57, 89): # F5-C8
  for n in range(59, 89): # G5-C8
    freq = xylo.tuning.yamaha_YX500R.note_to_freq(n)
    name = xylo.tuning.yamaha_YX500R.note_to_name(n)
    # orig_l = xylo.tuning.yamaha_YX500R.note_to_length(n)
    # l = orig_l * coeff[0] + (orig_l ** 2) * coeff[1]
    l = convert_length4(n)
    print(f"*** note {name} ({n}): ***")
    print(f"    freq {freq}, length {l}")
    sweep_blank(xylo.woods.aluminium_6060T5, make_bar(l, d))
    sum_lengths += l
    count += 1

print(f"total length {sum_lengths}, count {count}")

*** depth 0.006 ***
*** note G5 (59): ***
    freq 787.5544668360599, length 0.196
[ 808.21381273 2213.59988268 4299.44721904]

*** note G#5 (60): ***
    freq 834.384892410617, length 0.192
[ 842.11877952 2305.84803633 4476.94089339]

*** note A5 (61): ***
    freq 884.0, length 0.188
[ 878.19907764 2403.96348976 4665.58034738]

*** note A#5 (62): ***
    freq 936.5653754136171, length 0.184
[ 916.64667478 2508.45373774 4866.31708367]

*** note B5 (63): ***
    freq 992.2564507054858, length 0.18
[ 957.67171774 2619.88366532 5080.20132089]

*** note C6 (64): ***
    freq 1051.2590896624054, length 0.17600000000000002
[1001.50978814 2738.87328511 5308.39481905]

*** note C#6 (65): ***
    freq 1113.770208107068, length 0.17200000000000001
[1048.42333564 2866.12531009 5552.19317359]

*** note D6 (66): ***
    freq 1179.9984310863103, length 0.168
[1098.70725198 3002.41427114 5813.04513392]

*** note D#6 (67): ***
    freq 1250.1647891378161, length 0.164
[1152.69135869 3148.61945742 609

In [26]:
# freq = xylo.tuning.yamaha_YX500R.note_to_freq(n)
# name = xylo.tuning.yamaha_YX500R.note_to_name(n)
# orig_l = xylo.tuning.yamaha_YX500R.note_to_length(n)
# l = orig_l * coeff[0] + (orig_l ** 2) * coeff[1]
l = 0.200
print(f"    length {l}")
sweep_blank(xylo.woods.aluminium_6060T5, make_bar(l, 0.006))

    length 0.2
[ 776.31424172 2126.76212507 4132.24510432]



In [43]:
sum_lengths = 0
count = 0
lengths = {}

# on test bar 200 F#5, the drilling and finishing reduced the frequency from 745 to 735. so, make the cut at 1.5% above the target frequency
overshoot = 1.015

d = 0.006
# for n in range(59, 89): # G5-C8
for n in range(57, 89): # F5-C8
  freq = xylo.tuning.yamaha_YX500R.note_to_freq(n)
  name = xylo.tuning.yamaha_YX500R.note_to_name(n)
  l, freq_est = sweep_find_length(xylo.woods.aluminium_6060T5, freq * overshoot, d)
  print(f"*** note {name} ({n}): ***")
  print(f"    target freq {freq}, length {l}, est cut freq {freq_est}")
  sum_lengths += l
  count += 1
  lengths[n] = l

print(f"total length {sum_lengths}, count {count}")


*** note F5 (57): ***
    target freq 701.6312649699441, length 0.20898437500000006, est cut freq 711.1981107314082
*** note F#5 (58): ***
    target freq 743.3524310842836, length 0.20292968750000007, est cut freq 755.5816237415597
*** note G5 (59): ***
    target freq 787.5544668360599, length 0.1970703125, est cut freq 797.9114093634538
*** note G#5 (60): ***
    target freq 834.384892410617, length 0.19140625000000003, est cut freq 847.3323248402262
*** note A5 (61): ***
    target freq 884.0, length 0.1859375, est cut freq 897.7151070982798
*** note A#5 (62): ***
    target freq 936.5653754136171, length 0.18066406250000006, est cut freq 948.6284840401653
*** note B5 (63): ***
    target freq 992.2564507054858, length 0.1755859375, est cut freq 1008.4512625064746
*** note C6 (64): ***
    target freq 1051.2590896624054, length 0.17050781250000002, est cut freq 1064.3415432775644
*** note C#6 (65): ***
    target freq 1113.770208107068, length 0.16562500000000002, est cut freq 1130

In [44]:
print(lengths)

{57: 0.20898437500000006, 58: 0.20292968750000007, 59: 0.1970703125, 60: 0.19140625000000003, 61: 0.1859375, 62: 0.18066406250000006, 63: 0.1755859375, 64: 0.17050781250000002, 65: 0.16562500000000002, 66: 0.16093749999999998, 67: 0.15625, 68: 0.15175781250000003, 69: 0.1474609375, 70: 0.14316406250000008, 71: 0.13925781250000008, 72: 0.13515625, 73: 0.13125000000000003, 74: 0.12753906250000008, 75: 0.12382812500000002, 76: 0.12011718750000006, 77: 0.11699218750000004, 78: 0.11347656250000004, 79: 0.11035156250000003, 80: 0.10703125000000004, 81: 0.10410156250000002, 82: 0.10097656250000003, 83: 0.09804687500000002, 84: 0.09511718750000003, 85: 0.0923828125, 86: 0.08964843750000001, 87: 0.0873046875, 88: 0.0845703125}


In [45]:
def mklens(lens, filename, offsetx = 1.3):
  d = drawsvg.Drawing('297mm', '210mm')
  d.append(drawsvg.Rectangle(0, 0, '297mm', '210mm', stroke='green', stroke_width=2, fill='none'))
  d.append(drawsvg.Text(filename, '5mm', '20mm', '8mm'))
  grp = drawsvg.Group()
  y = 10
  for k,v in lens.items():
    mm = v * 1000 + offsetx
    # print(k,v)
    grp.append(drawsvg.Text(f'{k}', '2mm', f'{mm-1}mm', f'{y + 3}mm'))
    grp.append(drawsvg.Circle(f'{mm}mm', f'{y + 1}mm', '0.5mm'))
    grp.append(drawsvg.Rectangle(f'{offsetx}mm', f'{y}mm', f'{mm}mm', f'0.5mm', stroke='red', stroke_width=1))
  grp.append(drawsvg.Circle(f'{offsetx}mm', f'{y+1}mm', '1mm'))

  d.append(grp)

  d.save_html(f'data/layout/{filename}.html')
  return d

mklens(lengths, 'glock-lengths')
